# Déchiffrage d'Emails avec Python
Dans cette Tech Review, nous présentons les différentes solutions étudiées pour déchiffrer des
emails chiffrés avec Python.  

## Import des librairies


In [1]:
import subprocess
from pprint import pprint

from asn1crypto import cms
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.asymmetric import padding as asymmetric_padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import Encoding, load_pem_private_key
from cryptography.hazmat.primitives.serialization.pkcs7 import (
    PKCS7EnvelopeBuilder,
    PKCS7Options,
)
from cryptography.x509 import load_pem_x509_certificate

## Lecture du certificat X.509

In [2]:
# Clé publique : certificat RSA
with open("vectors/rsa_ca.pem", "rb") as file:
    certificate = load_pem_x509_certificate(file.read())

In [3]:
# Clé privée : RSA
with open("vectors/rsa_key.pem", "rb") as file:
    private_key = load_pem_private_key(file.read(), password=None)

## Chiffrement d'un email

In [27]:
# Écriture d'un message non-chiffré
message = b"Hello world!\n"
with open("vectors/msg.txt", "wb") as file:
    file.write(message)

### Avec `openssl`

In [ ]:
# SMIME : ancienne version
instructions = [
    "openssl",
    "smime",
    "-encrypt",
    "-aes-128-cbc",
    "-in",
    "vectors/msg.txt",
    "-out",
    "vectors/enveloped.der",
    "-outform",
    "der",
    "vectors/rsa_ca.pem",
]
smime_encryption_result = subprocess.run(instructions, check=True, capture_output=True)

In [7]:
# CMS : nouvelle version
instructions = [
    "openssl",
    "cms",
    "-encrypt",
    "-in",
    "vectors/msg.txt",
    "-out",
    "vectors/enveloped-rsa-oaep.pem",
    "-outform",
    "PEM",
    "-aes-128-cbc",
    "-recip",
    "vectors/rsa_ca.pem",
    "-keyopt",
    "rsa_padding_mode:oaep",
]
cms_encryption_result = subprocess.run(instructions, check=True)

### Avec `cryptography`
Les fonctionnalités de chiffrage d'email sont disponibles dans la librairie `cryptography` depuis
début 2024. 

In [8]:
# Prepare the envelope builder
envelope_builder = PKCS7EnvelopeBuilder()
envelope_builder = envelope_builder.add_recipient(certificate)
envelope_builder = envelope_builder.set_data(message)

# Encrypt the data
enveloped = envelope_builder.encrypt(Encoding.DER, [PKCS7Options.Binary])

In [9]:
# Store the data in vectors
with open("vectors/enveloped.der", "wb") as file:
    file.write(enveloped)

## Solutions abordées

### Solution 1: avec `openssl`

#### Structure ASN.1

In [10]:
instructions = [
    "openssl",
    "asn1parse",
    "-in",
    "vectors/enveloped.der",
    "-inform",
    "der",
]
output = subprocess.run(instructions, check=True, capture_output=True)
print(output.stdout.decode())

    0:d=0  hl=4 l= 667 cons: SEQUENCE          
    4:d=1  hl=2 l=   9 prim: OBJECT            :pkcs7-envelopedData
   15:d=1  hl=4 l= 652 cons: cont [ 0 ]        
   19:d=2  hl=4 l= 648 cons: SEQUENCE          
   23:d=3  hl=2 l=   1 prim: INTEGER           :00
   26:d=3  hl=4 l= 579 cons: SET               
   30:d=4  hl=4 l= 575 cons: SEQUENCE          
   34:d=5  hl=2 l=   1 prim: INTEGER           :00
   37:d=5  hl=2 l=  39 cons: SEQUENCE          
   39:d=6  hl=2 l=  26 cons: SEQUENCE          
   41:d=7  hl=2 l=  24 cons: SET               
   43:d=8  hl=2 l=  22 cons: SEQUENCE          
   45:d=9  hl=2 l=   3 prim: OBJECT            :commonName
   50:d=9  hl=2 l=  15 prim: UTF8STRING        :cryptography CA
   67:d=6  hl=2 l=   9 prim: INTEGER           :E712D3A0A56ED6C9
   78:d=5  hl=2 l=  13 cons: SEQUENCE          
   80:d=6  hl=2 l=   9 prim: OBJECT            :rsaEncryption
   91:d=6  hl=2 l=   0 prim: NULL              
   93:d=5  hl=4 l= 512 prim: OCTET STRING      [HEX 

#### Déchiffrement

In [4]:
instructions = [
    "openssl",
    "smime",
    "-decrypt",
    "-in",
    "vectors/enveloped.der",
    "-inkey",
    "vectors/rsa_key.pem",
    "-inform",
    "der",
]
output = subprocess.run(instructions, capture_output=True)
output.stdout.decode()

'Hello, World!\r\n'

### Solution 2 : avec `asn1crypto`

In [5]:
with open("vectors/enveloped.der", "rb") as file:
    enveloped = file.read()

#### Structure ASN.1

In [6]:
# Load the structure
content_info = cms.ContentInfo.load(enveloped)
content_type: cms.ContentType = content_info["content_type"]
enveloped_data: cms.EnvelopedData = content_info["content"]

# About the content type
content_type.native, content_type.dotted

('enveloped_data', '1.2.840.113549.1.7.3')

In [7]:
# Nested ordered dict to dict
def ordered_dict_to_dict(ordered_dict):
    return {
        key: ordered_dict_to_dict(value) if isinstance(value, dict) else value
        for key, value in ordered_dict.items()
    }

In [8]:
# About the enveloped data
# -- Version
version: cms.CMSVersion = enveloped_data["version"]
print("Version:", version.native)

# -- Originator info
originator_info: cms.OriginatorInfo = enveloped_data["originator_info"]

# -- Recipient infos
recipient_infos: cms.RecipientInfos = enveloped_data["recipient_infos"]
for i, recipient_info in enumerate(recipient_infos):
    print(f"\nRecipient {i}:")
    pprint(ordered_dict_to_dict(recipient_info.native))

# -- Encrypted content info
encrypted_content_info: cms.EncryptedContentInfo = enveloped_data["encrypted_content_info"]
print("\nEncrypted content info:")
pprint(ordered_dict_to_dict(encrypted_content_info.native))

Version: v0

Recipient 0:
{'encrypted_key': b"\x08\x08e\x84\xf1\xddCl\xda\x1f\xb5'\xb2C\xfa\x02"
                  b'\xa5\xb0\x10QG\xe1\xe9\x98m\xf5F/\xe4\x17\xd9\xd0\r&B7'
                  b';\xd2\xa1\x06\xae\x98;\xf3\xec\xca\xa0\xf9\xa2\x92\xb4\x08'
                  b'-\x99\xc6\xfd\x89i\xa4\x8d\xfb\x9dzt\xea\xab4\xba'
                  b'\xbb\x0c\xbc\x94}\xca0aGD\x1bY\xabI\x0f\xdc\x8dV\xf4>'
                  b'\x81^^\x053m\\\xd0\xcd\xe1\x94\x8e\xc2\xc6Pw\x1f\x86[\x0f'
                  b':t<\x9e\x9b%c\x06{\xbd\xcc\xed*\xb4\xc0j\xa1M)\xd0'
                  b'u|\xf8\x92z\xe2)\xab!:\x16\x95\xf4\x85\x9d\xbf\xd0\xe87M'
                  b'A\x95\\<"I?k\xccD\x0fcp\x9c\xee\xe2\x8aF\x08)a/\x8e\x1b'
                  b't\xc2\xc5\xaa\x87\xa8\x12U\xf8z\xf0}\x8cT\xea\xc8}S\x87\x0c'
                  b'\xbd\xd8\xe0\xe6\x83\x03\x96\xd25\xa3\xfc\x0c\x13\x15r\xe7'
                  b'\r\x97+\x11\xffb\xa6\x1b\xfb\x911R\xafZ\xf4\xea\xe4pz\xfe'
                  b'\xdce=\xc5M;\xa2\x80\x19N\x1dA& 0Y\

#### Déchiffrement
Merci aux liens : 
- https://www.askpython.com/python/examples/implementing-aes-with-padding 
- https://stackoverflow.com/questions/44662262

In [9]:
# Decrypt the keys of recipient information, if possible
decrypted_key = None
for recipient_info in recipient_infos.native:
    serial_number = recipient_info["rid"]["serial_number"]
    if serial_number == certificate.serial_number:
        print("Serial number:", recipient_info["rid"]["serial_number"])
        print("Encrypted key size:", len(recipient_info["encrypted_key"]))
        decrypted_key = private_key.decrypt(
            recipient_info["encrypted_key"], asymmetric_padding.PKCS1v15()
        )
        print("Decrypted key size:", len(decrypted_key))
        break

Serial number: 16650603459265877705
Encrypted key size: 512
Decrypted key size: 16


In [24]:
# Check sizes of content
encrypted = encrypted_content_info.native
print("Algorithm:", encrypted["content_encryption_algorithm"]["algorithm"])
print("Parameters:", encrypted["content_encryption_algorithm"]["parameters"])
print("Parameters size:", len(encrypted["content_encryption_algorithm"]["parameters"]))
print("Encrypted content size:", len(encrypted["encrypted_content"]))

Algorithm: aes192_cbc
Parameters: b'\xab\x81!Bn\x96Ca;\xd3v\x8c\xd6U3\x84'
Parameters size: 16
Encrypted content size: 16


In [25]:
# Decrypt the content (AES 128 CBC)
def decrypt(ciphertext, key, initialization_vector) -> str:
    cipher = Cipher(algorithms.AES(key), modes.CBC(initialization_vector))
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(len(key) * 8).unpadder()
    plaintext = unpadder.update(padded_data) + unpadder.finalize()

    return plaintext.decode()

In [26]:
decrypted_content = decrypt(
    encrypted["encrypted_content"],
    decrypted_key,
    encrypted["content_encryption_algorithm"]["parameters"],
)
print("Decrypted content:", decrypted_content)

Decrypted content: Hello, World!



### Solution 3 : avec `cryptography` !
Nous avons ouvert une Pull Request pour ajouter la fonctionnalité de déchiffrement d'email dans la
librairie `cryptography`. Le lien vers la PR est
[ici](https://github.com/pyca/cryptography/pull/11555).

Cette nouvelle fonctionnalité sera bientôt disponible entre fin 2224 et début 2025.